In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 1. Clone yolov7 


In [ ]:
# Clone source code YOLOv7 về thư mục OD_train   
%cd /content/drive/MyDrive/OD_train
!git clone https://github.com/augmentedstartups/yolov7.git

#2. Download required packages

In [ ]:
# Cài đặt các thư viện cần thiết để train YOLOv7
%cd /content/drive/MyDrive/OD_train/yolov7
!pip install -r requirements.txt

#3. Download pretrained model

In [ ]:
# Tải weight pretrain 
%cd /content/drive/MyDrive/OD_train/yolov7
!mkdir pretrain
%cd pretrain
!rm yolov7-e6.pt
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6.pt

#4. Prepare dataset

### Clone soucre Data

In [ ]:
# Clone source Data về thư mục OD_train   
%cd /content/drive/MyDrive/OD_train
!git clone https://github.com/dotrannhattuong/Dataset_OD_SelfDrivingCar

### Visualize Data

In [ ]:
from sklearn.model_selection import train_test_split
import glob
import os
from shutil import copyfile

In [ ]:
import altair as alt
import pandas as pd
import numpy as np

In [ ]:
lst = []
for file in glob.glob('/content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/yolo_data_detect/train/labels/*.txt'):
  df = pd.read_csv(file, sep=" ", header=None, names=['class', 'x', 'y', 'w', 'h'])
  lst.append(df)
signs = pd.concat(lst)
print(signs)

    class         x         y         w         h
0       1  0.316406  0.290278  0.039062  0.063889
0       1  0.331250  0.291667  0.034375  0.055556
0       1  0.334375  0.290278  0.031250  0.052778
0       1  0.335938  0.294444  0.031250  0.055556
0       1  0.337500  0.294444  0.031250  0.055556
..    ...       ...       ...       ...       ...
0       6  0.551562  0.430556  0.090625  0.100000
0       6  0.552344  0.429167  0.089063  0.102778
0       6  0.550781  0.433333  0.085938  0.094444
0       6  0.550781  0.431944  0.085938  0.102778
0       6  0.552344  0.434722  0.079687  0.097222

[3703 rows x 5 columns]


In [ ]:
data = pd.DataFrame()
data['counts'] = signs['class'].value_counts()
data.index = ['turn_right', 'straight', 'no_turn_left', 'no_turn_right', 'no_straight', 'car', 'unknown', 'turn_left']
data['title'] = data.index
print(data)

               counts          title
turn_right        644     turn_right
straight          539       straight
no_turn_left      498   no_turn_left
no_turn_right     486  no_turn_right
no_straight       454    no_straight
car               366            car
unknown           363        unknown
turn_left         353      turn_left


In [ ]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('counts', axis=alt.Axis(title='Counts')),
    y=alt.Y('title',
        sort=alt.EncodingSortField(field='counts', order='ascending', op='sum'),
        axis=alt.Axis(title='Title')
    ),
    color=alt.Color('title:N')
)

alt.Chart(...)

#5. Prepare custom .yaml file


In [ ]:
# Khai báo 1 file yaml để YOLOv7 biết:
# - Đường dẫn đến thư mục train, test (nếu có, nếu không thì dùng luôn đường dẫn đến train)
# - Số lượng class qua biến nc (number of class)
# - Tên của các class
%cd /content/drive/MyDrive/OD_train/yolov7
!rm data/mydataset.yaml # nếu có
!echo 'train: /content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/yolo_data_detect/train' >> data/mydataset.yaml
!echo 'val: /content/drive/MyDrive/OD_train/Dataset_OD_SelfDrivingCar/yolo_data_detect/val' >> data/mydataset.yaml
!echo 'nc: 9' >> data/mydataset.yaml
!echo "names: ['no', 'turn_right', 'straight', 'no_turn_left', 'no_turn_right', 'no_straight', 'car', 'unknown', 'turn_left']" >> data/mydataset.yaml

# 6. Train

### First Training

In [ ]:
# Train model YOLOv7 với dữ liệu OD
%cd /content/drive/MyDrive/CD_train/yolov7
!python train_aux.py --batch 16 --cfg cfg/training/yolov7-e6.yaml --epochs 500 --data data/mydataset.yaml --weights 'pretrain/yolov7-e6.pt' --hyp data/hyp.scratch.p5.yaml

### Resume 

In [ ]:
%cd /content/drive/MyDrive/CD_train/yolov7

!python train_aux.py --resume /content/drive/MyDrive/UTE2023/train/exp1/weights/last.pt --cache

#7. Inference

In [ ]:
#Nhận diện thử với weights vừa train
%cd /content/drive/MyDrive/OD_train/yolov7
!python detect.py --weights /content/drive/MyDrive/OD_train/exp2/weights/best.pt --source /content/drive/MyDrive/OD_train/test

In [ ]:
# Xem ảnh đã nhận diện
from IPython.display import Image, display
display(Image(filename="/content/drive/MyDrive/OD_train/yolov7/runs/detect/exp"))